# Exploring Convolutional Layers: Fashion-MNIST

## Description
This notebook explores the design and behavior of convolutional neural networks (CNNs) through systematic experimentation on the Fashion-MNIST dataset. We are going to analyze how architectural choices impact performance and learning.

## Objectives
1. **Dataset Exploration (EDA)**: Understand the structure, distribution, and characteristics of Fashion-MNIST
2. **Baseline Model**: Implement a fully connected network as a performance reference
3. **CNN Architecture Design**: Build a convolutional network with justified design decisions
4. **Controlled Experiments**: Systematically vary one architectural aspect (kernel size) and measure its impact
5. **Interpretation**: Explain why convolutions introduce useful inductive bias for image data


In [ ]:
#Install required libraries
%pip install numpy pandas matplotlib tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()